In [1]:
import pandas as pd
from matplotlib.font_manager import json_dump
from solana.transaction import Transaction

from src.solana_analyst import SolanaAnalyst

block_id = 285907174
solana_analyst = SolanaAnalyst()
transactions = solana_analyst.get_transactions(block_id)
filtered_transactions = solana_analyst.filter_signers(transactions, solana_analyst.signer1, solana_analyst.signer2)
df = pd.json_normalize(filtered_transactions)
df

,version,transaction.signatures,transaction.message.header.numRequiredSignatures,transaction.message.header.numReadonlySignedAccounts,transaction.message.header.numReadonlyUnsignedAccounts,transaction.message.accountKeys,transaction.message.recentBlockhash,transaction.message.instructions,transaction.message.addressTableLookups,meta.err,...,meta.innerInstructions,meta.logMessages,meta.preTokenBalances,meta.postTokenBalances,meta.rewards,meta.loadedAddresses.writable,meta.loadedAddresses.readonly,meta.returnData.programId,meta.returnData.data,meta.computeUnitsConsumed
0,0,[2hLh7zGoGV5m1SagCRk3PmmfwEa98ydiDP6vC2DiVHeiS...,2,0,5,"[Gm16UVXwhB8oaipRTa52qJ36zWeBK4k3CurkT8JGE5VE,...",J2ywbPt7vaskiFwRwHf2waAdCLDBa4ofTUYTfEWnTDJt,"[{'programIdIndex': 4, 'accounts': [], 'data':...",[{'accountKey': '7LPVJWJskPPzLtwkR3hW7QNZhcR8b...,None,...,"[{'index': 2, 'instructions': [{'programIdInde...",[Program ComputeBudget111111111111111111111111...,"[{'accountIndex': 10, 'mint': 'Ece6xH1aDi3y9w1...","[{'accountIndex': 10, 'mint': 'Ece6xH1aDi3y9w1...",[],"[vNjPNcf2HiSjUieEoBtVRP96yeMH1qrUN2hgxh7ZUzr, ...","[11111111111111111111111111111111, TokenkegQfe...",AzHrwdCsEZotAjr7sjenHrHpf1ZKYoGBP6N7HVhEsyen,"[0eQWAwAAAAA=, base64]",174098


In [8]:
preTokenBalances = df['meta.preTokenBalances'][0]


In [9]:
postTokenBalances = df['meta.postTokenBalances'][0]

In [10]:
def extract_data(balances, balance_type):
    extracted_data = []
    for balance in balances:
        extracted_data.append({
            'accountIndex': balance['accountIndex'],
            'mintToken': balance['mint'],
            'owner': balance['owner'],
            f'{balance_type}Balance': float(balance['uiTokenAmount']['uiAmount']) if balance['uiTokenAmount'][
                                                                                         'uiAmount'] is not None else 0
        })
    return pd.DataFrame(extracted_data)


df_pre = extract_data(preTokenBalances, 'pre')
df_post = extract_data(postTokenBalances, 'post')

# Hợp nhất dữ liệu pre và post theo accountIndex và mintToken
df_combined = pd.merge(df_pre, df_post, on=['accountIndex', 'mintToken', 'owner'], how='outer')
df_combined['change'] = df_combined['postBalance'] - df_combined['preBalance']

# Hiển thị DataFrame kết quả
df_combined

,accountIndex,mintToken,owner,preBalance,postBalance,change
0,10,Ece6xH1aDi3y9w1BZViwJiZeZ52j96Z8aEr16daGpump,3r8txc9CQJYe1oScR1vYnohmcVRmKG2LtwhVeE2jnM9L,0.000000e+00,0.000000e+00,0.000000
1,13,Ece6xH1aDi3y9w1BZViwJiZeZ52j96Z8aEr16daGpump,5Q544fKrFoe6tsEbD7S8EmxGTJYAKtTVhAW5Q5pge4j1,1.092027e+08,1.088740e+08,-328765.260581
2,15,So11111111111111111111111111111111111111112,3r8txc9CQJYe1oScR1vYnohmcVRmKG2LtwhVeE2jnM9L,0.000000e+00,0.000000e+00,0.000000
3,16,So11111111111111111111111111111111111111112,5Q544fKrFoe6tsEbD7S8EmxGTJYAKtTVhAW5Q5pge4j1,1.613291e+02,1.618175e+02,0.488384
4,22,Ece6xH1aDi3y9w1BZViwJiZeZ52j96Z8aEr16daGpump,tyqivnLqDzhheejZYyYZ4tKTmSwCNJ4CwTuRAZAucnL,2.468297e+06,2.797062e+06,328765.260581
5,23,So11111111111111111111111111111111111111112,tyqivnLqDzhheejZYyYZ4tKTmSwCNJ4CwTuRAZAucnL,4.503115e+00,3.962899e+00,-0.540216


In [35]:
import binascii
from base58 import b58decode
import json
from solana.rpc.api import Client


def decode_solana_transaction(tx_signature):
    # Lấy chi tiết giao dịch bằng Transaction Signature
    transaction_info = filtered_transactions[0]

    # Lấy phần message của giao dịch
    message = transaction_info['transaction']['message']

    # Lấy danh sách các tài khoản liên quan (accountKeys)
    account_keys = message['accountKeys']

    # Lấy danh sách các lệnh (instructions)
    instructions = message['instructions']

    # Lặp qua từng lệnh để giải mã dữ liệu
    for i, instruction in enumerate(instructions):
        program_id = account_keys[instruction['programIdIndex']]
        data = instruction['data']
        decoded_data = b58decode(data)  # Giải mã từ base64

        print(f"Instruction {i + 1}:")
        print(f"  Program ID: {program_id}")
        print(f"  Decoded Data (hex): {decoded_data.hex()}")

        # Kiểm tra nếu có tài khoản liên quan
        if instruction['accounts']:
            print("  Accounts Involved:")
            for account_index in instruction['accounts']:
                account = account_keys[account_index]
                print(f"    - {account}")

        print()


# Ví dụ sử dụng hàm
tx_signature = "2hLh7zGoGV5m1SagCRk3PmmfwEa98ydiDP6vC2DiVHeiSTpv1KAM5DzwNefXQVwxrNSoBJGpznvYY9SoqzJ5qv9V"
decode_solana_transaction(tx_signature)



Instruction 1:
  Program ID: ComputeBudget111111111111111111111111111111
  Decoded Data (hex): 03644d030000000000

Instruction 2:
  Program ID: ComputeBudget111111111111111111111111111111
  Decoded Data (hex): 028fcf0200

Instruction 3:
  Program ID: AzHrwdCsEZotAjr7sjenHrHpf1ZKYoGBP6N7HVhEsyen
  Decoded Data (hex): 030000000000000000080a033400000000f2443b1d00000000a50000000000000006ddf6e1d765a193d9cbe146ceeb79ac1cb485ed5f5b37913a8cf5857eff00a90a0501010a07010101131b0f010000000000000000090000000000000000000000000000000001132204080000000000000000f8c69e91e17587c8000000000000000000000000000000000a0401090a04010904030100
  Accounts Involved:


IndexError: list index out of range

In [57]:
import struct
from construct import Struct, Int8ul, Int64ul


def decode_instruction_data(encoded_data):
    # Define the structure of the instruction data
    instruction_format = Struct(
        "discriminator" / Int8ul,
        "amountIn" / Int64ul,
        "minimumAmountOut" / Int64ul
    )

    # Decode the instruction data
    decoded_data = instruction_format.parse(encoded_data)
    print('decoded_data', decoded_data)

    # Create the structured output
    result = {
        "discriminator": {
            "type": "u8",
            "data": decoded_data.discriminator
        },
        "amountIn": {
            "type": "u64",
            "data": decoded_data.amountIn
        },
        "minimumAmountOut": {
            "type": "u64",
            "data": decoded_data.minimumAmountOut
        }
    }

    return result


transaction_info = filtered_transactions[0]

# Lấy phần message của giao dịch
message = transaction_info['transaction']['message']

# Lấy danh sách các tài khoản liên quan (accountKeys)
account_keys = message['accountKeys']

# Lấy danh sách các lệnh (instructions)
instructions = message['instructions']

# Example usage
transaction_data = {
    "transaction": {
        "message": {
            "instructions": [
                {
                    "programIdIndex": 8,
                    "accounts": [30, 0, 24, 0, 1, 25, 1, 30, 0, 5, 7, 0, 2, 0, 33, 24, 25, 28, 25, 11, 27, 12, 18, 16,
                                 13, 31, 19, 9, 17, 14, 15, 10, 26, 1, 2, 0, 29, 21, 29, 22, 23, 2, 1, 33, 30, 20, 29,
                                 0, 25, 25, 32, 29, 3, 3, 3, 25, 1, 0, 0, 25, 2, 0, 0, 0, 24, 6],
                    "data": "8AYa3keV52j9yf3CFVdm31afDqz8GQLrB3RV1e2yjtaghYoNEtitKJAL4ixYAHPhP7FH3AXDdr9JgHJp49UPQ2zap8rGJNBiWb9KB7UJLsmeh8YN1iEFsZ13BCg6DMWvxB6fjAuXar3UF74ERbexvjCsRMJaBAu6RD7UNJKjUoUZoFXxp5iDhh46SzyKybwnS7hGoR4MZK5rUTh"
                }
            ]
        }
    }
}

# Get the instruction data
instruction_data = transaction_data["transaction"]["message"]["instructions"][0]["data"]

# Convert the instruction data from base58 to bytes
from base58 import b58decode

instruction_bytes = b58decode(instruction_data)

# Decode and print the result
result = decode_instruction_data(instruction_bytes)
print(result)

decoded_data Container: 
    discriminator = 3
    amountIn = 0
    minimumAmountOut = 872614408
{'discriminator': {'type': 'u8', 'data': 3}, 'amountIn': {'type': 'u64', 'data': 0}, 'minimumAmountOut': {'type': 'u64', 'data': 872614408}}


In [60]:
!pip install construct

In [69]:
import base58
import json
import struct

# Đọc file IDL
with open('/home/infcapital/Documents/DA_Blockchain/data/idl_files/raydium_v4_anchor_idl.json', 'r') as f:
    idl = json.load(f)
with open('/home/infcapital/Documents/DA_Blockchain/data/transaction_of_signer.json','r') as f:
    transaction_data = json.load(f)

# Hàm để decode một tham số
def decode_argument(arg_type, data):
    try:
        if arg_type == 'u8':
            return struct.unpack('B', data[:1])[0], data[1:]
        elif arg_type == 'u16':
            return struct.unpack('<H', data[:2])[0], data[2:]
        elif arg_type == 'u32':
            return struct.unpack('<I', data[:4])[0], data[4:]
        elif arg_type == 'u64':
            return struct.unpack('<Q', data[:8])[0], data[8:]
        elif arg_type == 'i64':
            return struct.unpack('<q', data[:8])[0], data[8:]
        elif arg_type == 'publicKey':
            return data[:32].hex(), data[32:]
        else:
            return f"Unsupported type: {arg_type}", data
    except struct.error:
        return f"Not enough data for {arg_type}", b''


# Hàm chính để decode instruction
def decode_instruction(instruction_data, idl):
    try:
        # Decode từ Base58
        decoded_data = base58.b58decode(instruction_data)

        # Decode discriminant (byte đầu tiên)
        discriminant, remaining_data = decode_argument('u8', decoded_data)

        # Tìm instruction tương ứng trong IDL
        matching_instruction = next(
            (instr for instr in idl['instructions'] if instr['discriminant']['value'] == discriminant), None)

        if not matching_instruction:
            return f"Unknown instruction with discriminant {discriminant}"

        # Decode các tham số
        decoded_args = {}
        for arg in matching_instruction['args']:
            value, remaining_data = decode_argument(arg['type'], remaining_data)
            decoded_args[arg['name']] = value
            if not remaining_data:
                break  # Dừng nếu hết dữ liệu

        return {
            'name': matching_instruction['name'],
            'args': decoded_args
        }
    except Exception as e:
        return f"Error decoding instruction: {str(e)}"


# Decode các instructions
for idx, instr in enumerate(transaction_data['transaction']['message']['instructions']):
    print(f"Instruction {idx}:")
    decoded = decode_instruction(instr['data'], idl)
    print(json.dumps(decoded, indent=4))
    print()

Instruction 0:
{
    "name": "deposit",
    "args": {
        "maxCoinAmount": 216420
    }
}

Instruction 1:
{
    "name": "monitorStep",
    "args": {
        "planOrderLimit": 53135,
        "placeOrderLimit": 2
    }
}

Instruction 2:
{
    "name": "deposit",
    "args": {
        "maxCoinAmount": 0,
        "maxPcAmount": 872614408,
        "baseSide": 490423538
    }
}

Instruction 3:
{
    "name": "initialize",
    "args": {
        "nonce": 0,
        "openTime": 32140396986368
    }
}

Instruction 4:
{
    "name": "initialize2",
    "args": {
        "nonce": "Not enough data for u8"
    }
}

Instruction 5:
{
    "name": "initialize2",
    "args": {
        "nonce": "Not enough data for u8"
    }
}

Instruction 6:
"Unknown instruction with discriminant 21"

Instruction 7:
{
    "name": "initialize",
    "args": {
        "nonce": 0,
        "openTime": 133646254080
    }
}

Instruction 8:
"Unknown instruction with discriminant 22"

Instruction 9:
"Unknown instruction with disc